In [474]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
import plot_utils 
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')
rsi = 0

In [666]:
df = pd.read_csv('BTC_5min_10AUG.csv')


In [667]:
def sma(data, window): 
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band

symbols = ['BTC']

nstd = 3
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

        
    



for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'],20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)        
    df[f'{symbol}_rsi'] = ta.RSI(df[f'{symbol}-USD_Close'])

        
df.dropna(inplace=True)

In [674]:
class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount
        self.balance_unit = balance_unit
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier # VIP level 0, paying fees with BNB
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append([self.balance_unit, time, sell_price])
        self.balance_unit = 'USDT'

env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=1)#0.99925
        

In [676]:

buy_price_1 = 0
rsi = 0
env.balance_amount = 100
for i in range(len(df)):
    
    #means looking for something to buy
    if df[f'{symbol}-USD_High'].iloc[i] > df[f'{symbol}_upper_band'].iloc[i]:
        if df[f'{symbol}_rsi'].iloc[i] > rsi:
            rsi = df[f'{symbol}_rsi'].iloc[i]
    if df[f'{symbol}-USD_High'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
        rsi = 0
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] < 25: #buy signal
                if rsi < 70:
                    env.buy(symbol, df[f'{symbol}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
                    break
    
    #means looking to sell
    if env.balance_unit != 'USDT':
        if df[f'{symbol}-USD_Close'].iloc[i] / buy_price_1 < 0.95:
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
            
            
        elif df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i] and df[f'{symbol}_rsi'].iloc[i] > 65: #sell signal
            env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[i], df['OpenTime'].iloc[i])
                    
    
if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])
    



In [677]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 62
num sells: 62
ending balance: 1.1796308801901802e+135 USDT


In [678]:
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    print(env.balance_amount)
    print('\n')

['BTC', '2022-08-19 16:35:00', 21872.8]
['BTC', '2022-08-20 02:35:00', 21600.06]
P/L = -1.2469368347902354
1.1796308801901802e+135


['BTC', '2022-08-23 15:30:00', 20969.63]
['BTC', '2022-08-24 00:05:00', 21640.09]
P/L = 3.1972905578209865
1.1796308801901802e+135


['BTC', '2022-08-26 21:10:00', 21201.98]
['BTC', '2022-08-26 22:30:00', 21561.5]
P/L = 1.695690685492579
1.1796308801901802e+135


['BTC', '2022-08-27 09:15:00', 20209.71]
['BTC', '2022-08-28 12:30:00', 20121.93]
P/L = -0.43434566849300804
1.1796308801901802e+135


['BTC', '2022-08-29 09:25:00', 19713.43]
['BTC', '2022-08-29 12:25:00', 19776.7]
P/L = 0.3209487136434319
1.1796308801901802e+135


['BTC', '2022-09-04 05:10:00', 19693.2]
['BTC', '2022-09-04 19:05:00', 19816.48]
P/L = 0.6260028842443033
1.1796308801901802e+135


['BTC', '2022-09-07 03:10:00', 19175.76]
['BTC', '2022-09-07 23:00:00', 18824.33]
P/L = -1.832678339737237
1.1796308801901802e+135


['BTC', '2022-09-13 22:35:00', 21798.43]
['BTC', '2022-09-14 12:20:00',

1.1796308801901802e+135

0
1
2
